# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [93]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


from sklearn.pipeline import Pipeline, FeatureUnion
# import numpy as np
from sklearn.datasets import make_multilabel_classification
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report

import nltk
nltk.download('punkt')
nltk.download('wordnet')
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ieva\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ieva\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [24]:
import sqlalchemy

In [25]:
engine = create_engine('sqlite:///DisasterResponse.db')


In [26]:
engine.table_names()

['data_prep']

In [27]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('data_prep', engine)  
X = df.message
Y = df.drop(['message', 'id', 'original', 'genre'], axis=1)
category_names = Y.columns.tolist()


In [28]:
df.head(5)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
X.head(5)

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [30]:
Y.head(5)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [31]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [32]:
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer = tokenize)),
                ('tfidf', TfidfTransformer())
            ]))#,

#             ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(KNeighborsClassifier()))
    ])

In [ ]:
# def build_model():
#     pipeline = Pipeline([
#         ('features', FeatureUnion([

#             ('text_pipeline', Pipeline([
#                 ('vect', CountVectorizer(tokenizer=tokenize)),
#                 ('tfidf', TfidfTransformer())
#             ]))
#         ])),

#         ('clf', MultiOutputClassifier(KNeighborsClassifier()))
#     ])

#     parameters = [
#         {"clf": [MultiOutputClassifier(RandomForestClassifier())],
#         "clf__estimator__n_estimators": [10, 100],
#         "clf__estimator__max_depth":[8]},
#         {"clf": [MultiOutputClassifier(KNeighborsClassifier())],
#         "clf__estimator__leaf_size": [5, 10],
#         "clf__estimator__n_neighbors": [3, 5]}
#     ]

#     cv = GridSearchCV(pipeline, param_grid=parameters, cv=3, n_jobs=-1)
#     return cv

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [47]:
    X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 1, test_size = 0.3)
    model = pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [34]:
    y_pred = model.predict(X_test)


In [35]:
len(category_names)

36

In [36]:
y_pred[:,0]

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

In [37]:
    for i in range(len(category_names)):
        print("Category:", category_names[i],"\n", classification_report(y_test.iloc[:, i].values, y_pred[:, i]))

Category: related 
               precision    recall  f1-score   support

           0       0.54      0.41      0.46      1550
           1       0.82      0.89      0.86      4951
           2       0.79      0.21      0.33        53

    accuracy                           0.77      6554
   macro avg       0.72      0.50      0.55      6554
weighted avg       0.76      0.77      0.76      6554

Category: request 
               precision    recall  f1-score   support

           0       0.88      0.98      0.93      5415
           1       0.81      0.35      0.49      1139

    accuracy                           0.87      6554
   macro avg       0.84      0.67      0.71      6554
weighted avg       0.87      0.87      0.85      6554

Category: offer 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6554
   macro avg    

C:\Users\Ieva\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Category: money 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      6413
           1       0.50      0.03      0.05       141

    accuracy                           0.98      6554
   macro avg       0.74      0.51      0.52      6554
weighted avg       0.97      0.98      0.97      6554

Category: missing_people 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6474
           1       0.00      0.00      0.00        80

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

Category: refugees 
               precision    recall  f1-score   support

           0       0.97      1.00      0.98      6338
           1       0.25      0.02      0.03       216

    accuracy                           0.97      6554
   macro avg       0.61      0.51      0.51      6554
weighted

### 6. Improve your model
Use grid search to find better parameters. 

In [55]:
parameters = [
    {"clf": [MultiOutputClassifier(KNeighborsClassifier())], 
    'clf__n_neighbors': [3, 5, 10]
    }, 
    {"clf": [MultiOutputClassifier(KNeighborsClassifier())], 
    'clf__max_item': [50]
    },     
    {"clf": [MultiOutputClassifier(RandomForestClassifier())],
    "clf__estimator__n_estimators": [10, 50, 100],
    "clf__estimator__max_depth":[3, 5]}
]


cv = GridSearchCV(pipeline, param_grid = parameters, n_jobs = -1, cv = 4)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [39]:
cv.fit(X_train, y_train)

C:\Users\Ieva\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('features',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('text_pipeline',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('vect',
                                                                                         CountVectorizer(analyzer='word',
                                                                                                         binary=False,
                                                                                                         decode_error='strict',
                                                                                                         dtype=<class 'numpy.int64'

In [41]:
y_pred = cv.predict(X_test)

In [40]:
cv

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('features',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('text_pipeline',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('vect',
                                                                                         CountVectorizer(analyzer='word',
                                                                                                         binary=False,
                                                                                                         decode_error='strict',
                                                                                                         dtype=<class 'numpy.int64'

In [45]:
sorted(cv.cv_results_.keys())

['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'param_clf__estimator__leaf_size',
 'param_clf__estimator__n_neighbors',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split1_test_score',
 'split2_test_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score']

In [46]:
    for i in range(len(category_names)):
        print("Category:", category_names[i],"\n", classification_report(y_test.iloc[:, i].values, y_pred[:, i]))

Category: related 
               precision    recall  f1-score   support

           0       0.54      0.41      0.46      1550
           1       0.82      0.89      0.86      4951
           2       0.79      0.21      0.33        53

    accuracy                           0.77      6554
   macro avg       0.72      0.50      0.55      6554
weighted avg       0.76      0.77      0.76      6554

Category: request 
               precision    recall  f1-score   support

           0       0.88      0.98      0.93      5415
           1       0.81      0.35      0.49      1139

    accuracy                           0.87      6554
   macro avg       0.84      0.67      0.71      6554
weighted avg       0.87      0.87      0.85      6554

Category: offer 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6554
   macro avg    

C:\Users\Ieva\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


 buildings 
               precision    recall  f1-score   support

           0       0.95      0.99      0.97      6199
           1       0.45      0.08      0.13       355

    accuracy                           0.94      6554
   macro avg       0.70      0.54      0.55      6554
weighted avg       0.92      0.94      0.93      6554

Category: electricity 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      6417
           1       0.78      0.05      0.10       137

    accuracy                           0.98      6554
   macro avg       0.88      0.53      0.54      6554
weighted avg       0.98      0.98      0.97      6554

Category: tools 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6516
           1       0.00      0.00      0.00        38

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [49]:
filename = 'tmp.sav'
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [9]:
import sys
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.datasets import make_multilabel_classification
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report

import nltk
nltk.download('punkt')
nltk.download('wordnet')
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ieva\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ieva\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
def load_data(database_filepath):
    engine = create_engine('sqlite:///' + database_filepath)
    df = pd.read_sql_table('data_prep', engine)  
    X = df.message
    Y = df.drop(['message', 'id', 'original', 'genre'], axis=1)
    category_names = Y.columns.tolist()

    return X, Y, category_names 


In [11]:
database_filepath = "DisasterResponse.db"

In [12]:
X, Y, category_names = load_data(database_filepath)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)


In [13]:
X

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
                               ...                        
26211    The training demonstrated how to enhance micro...
26212    A suitable candidate has been selected and OCH...
26213    Proshika, operating in Cox's Bazar municipalit...
26214    Some 2,000 women protesting against the conduc...
26215    A radical shift in thinking came about as a re...
Name: message, Length: 26216, dtype: object

In [14]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

In [15]:
def build_model():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer = tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
        ])),

        ('clf', MultiOutputClassifier(KNeighborsClassifier()))
    ])

    parameters = [
#         {"clf": [MultiOutputClassifier(KNeighborsClassifier())], 
#         'clf__estimator__n_neighbors': [3]
#         }, 
        {"clf": [MultiOutputClassifier(DecisionTreeClassifier())], 
        'clf__estimator__max_depth': [3]
        },     
#         {"clf": [MultiOutputClassifier(RandomForestClassifier())],
#         "clf__estimator__n_estimators": [10],
#         "clf__estimator__max_depth":[3]}
    ]

    cv = GridSearchCV(pipeline, param_grid = parameters, n_jobs = -1, cv = 4)
    return cv

In [16]:
        model = build_model()


In [17]:
        model.fit(X_train, Y_train)


GridSearchCV(cv=4, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('features',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('text_pipeline',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('vect',
                                                                                         CountVectorizer(analyzer='word',
                                                                                                         binary=False,
                                                                                                         decode_error='strict',
                                                                                                         dtype=<class 'numpy.int64'>,
  

In [18]:
def evaluate_model(model, X_test, Y_test, category_names):
    y_pred = model.predict(X_test)
    for i in range(len(category_names)):
        print("Category:", category_names[i],"\n", classification_report(Y_test.iloc[:, i].values, y_pred[:, i]))


In [19]:
evaluate_model(model, X_test, Y_test, category_names)


C:\Users\Ieva\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Category: related 
               precision    recall  f1-score   support

           0       0.66      0.03      0.06      1245
           1       0.76      0.99      0.86      3964
           2       0.00      0.00      0.00        35

    accuracy                           0.76      5244
   macro avg       0.47      0.34      0.31      5244
weighted avg       0.73      0.76      0.67      5244

Category: request 
               precision    recall  f1-score   support

           0       0.88      0.98      0.93      4384
           1       0.75      0.33      0.46       860

    accuracy                           0.87      5244
   macro avg       0.81      0.65      0.69      5244
weighted avg       0.86      0.87      0.85      5244

Category: offer 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      5224
           1       0.00      0.00      0.00        20

    accuracy                           1.00      5244
   macro avg    

In [20]:
def save_model(model, model_filepath):
    filename = model_filepath
    pickle.dump(model, open(filename, 'wb'))

In [106]:
        save_model(model, "test.sav")
